# Create mlflow experiment to test different classifier on their performance

## Used algorithms
- [x] LogisticRegression
- [x] DecisionTreeClassifier
- [x] ExtraTreesClassifier
- [x] GradientBoostingClassifier
- [x] RandomForestClassifier
- [x] SVC
- [x] xgboost
  - [ ] implement early stopping only for xgboost (maybe use a new Jupyter notebbok but write into the same mlflow experiment)
- [x] lightgbm
  - [ ] implement early stopping only for lightgbm (maybe use a new Jupyter notebbok but write into the same mlflow experiment)

In [1]:
import pandas as pd
print("pandas version: {}". format(pd.__version__))

# numpy: support for large, multi-dimensional arrays and matrices and high-level mathematical functions
import numpy as np
print("numpy version: {}". format(np.__version__))

import sklearn
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier, GradientBoostingClassifier, RandomForestClassifier
from sklearn.svm import SVC
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit, train_test_split, cross_val_score, learning_curve
from sklearn.feature_selection import VarianceThreshold
from sklearn.metrics import precision_recall_curve, roc_curve, confusion_matrix, roc_auc_score
print("sklearn version: {}". format(sklearn.__version__))

import xgboost
from xgboost import XGBClassifier
print("xgboost version: {}". format(xgboost.__version__))

import lightgbm
from lightgbm import LGBMClassifier
print("lightgbm version: {}". format(lightgbm.__version__))

import optuna
print("optuna version: {}". format(optuna.__version__))

import mlflow
from mlflow.utils.mlflow_tags import MLFLOW_PARENT_RUN_ID
from mlflow.tracking import MlflowClient
print("mlflow version: {}". format(mlflow.__version__))

import matplotlib.pyplot as plt
import seaborn as sns

from datetime import datetime
import warnings
warnings.simplefilter('ignore')

pandas version: 1.4.3
numpy version: 1.21.5
sklearn version: 1.1.1
xgboost version: 1.6.1
lightgbm version: 3.3.2


e:\programs\anaconda3\envs\mlflow_optuna\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


optuna version: 2.10.1
mlflow version: 1.28.0


In [2]:
RANDOM_STATE=42
N_TRAILS=50 # run for 50 runs
TIMEOUT=600 # run for max 10 minutes (if the last run at 9 minutes runs for 1h, the active run is not killed)

In [3]:
client = MlflowClient()
try:
    experiment = client.create_experiment("Titanic")
except:
    experiment = client.get_experiment_by_name("Titanic").experiment_id

parent_run = client.create_run(experiment_id=experiment)

In [4]:
def load_data():
    # load prepared training and test dataset
    df_train = pd.read_pickle('../03_dataCleaningPreparation/df_train_prepared_reduced.pkl')
    df_test = pd.read_pickle('../03_dataCleaningPreparation/df_test_prepared_reduced.pkl')

    # split the training and test dataset to the input features (x_train, x_test) and the survival class (y_train)
    y_train = df_train['Survived']
    x_train = df_train.drop(['Survived'], axis=1)
    x_test = df_test

    x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.3, stratify = y_train, random_state = 42)

    return x_train, y_train, x_validate, y_validate, x_test

x_train, y_train, x_validate, y_validate, x_test = load_data()

In [5]:
# only scale the features that are not one-hot-encoded [0,1]
transformer_num = [
    "Pclass",
    "Parch",
    "Family",
    "TNlen",
    "LeadingDigit",
    "Fare_bin",
    "Age_bin",
    ]



# define cross validation
cv = ShuffleSplit(
    n_splits = 5,
    test_size = 0.3,
    train_size = 0.7,
    random_state = RANDOM_STATE
    )

In [6]:
def create_model(trial):   

    child_run = client.create_run(
        experiment_id=experiment,
        tags={
            MLFLOW_PARENT_RUN_ID: parent_run.info.run_id
        }
    )

    ''' feature selection '''
    feature_selection = trial.suggest_categorical("feature_selection", ["none", "reduce"])
    client.log_param(child_run.info.run_id, "feature_selection", feature_selection)



    ''' columnprep '''
    columnprep__transformers_num = trial.suggest_categorical("columnprep__transformers_num", ["StandardScaler", "MinMaxScaler"])

    if columnprep__transformers_num == "StandardScaler":
        col_transform = ColumnTransformer(
                    transformers=[
                        ('num', StandardScaler(), transformer_num)
                    ], remainder='passthrough'
                )
    elif columnprep__transformers_num == "MinMaxScaler":
        col_transform = ColumnTransformer(
            transformers=[
                ('num', MinMaxScaler(), transformer_num)
            ], remainder='passthrough'
        )


    ''' reducedim '''
    reducedim__n_components = trial.suggest_float("reducedim__n_components", 0.7, 0.9)
    client.log_param(child_run.info.run_id, "reducedim__n_components", reducedim__n_components)


    ''' algo '''
    model_type = trial.suggest_categorical('model_type',
        [
            'logistic-regression',
            'decision-tree',
            # 'svm',
            'xgb',
            'lgb',
            # 'extra_tree',
            'randomforest-classifier',
            # 'gradient-boosting'
            ]
        )


    if model_type == 'svm':
        svm_kernel = trial.suggest_categorical('svm_kernel', ['linear', 'poly', 'rbf', 'sigmoid'])
        svm_C = trial.suggest_float('svm_C', 1, 500, log=True)
        svm_degree = trial.suggest_discrete_uniform('svm_degree', 1, 5, 1)
        
        model = SVC(
            kernel=svm_kernel,
            C=svm_C,
            degree=svm_degree,
            probability=True,
            random_state=RANDOM_STATE
        )

        client.log_param(child_run.info.run_id, "svm_kernel", svm_kernel)
        client.log_param(child_run.info.run_id, "svm_C", svm_C)
        client.log_param(child_run.info.run_id, "svm_degree", svm_degree)
    

    if model_type == 'logistic-regression':
        lr_C = trial.suggest_float("lr_C", 1, 500, log=True)
        lr_penalty = trial.suggest_categorical('lr_penalty', ['l2', 'l1'])
        if lr_penalty == 'l1':
            lr_solver = 'saga'
        else:
            lr_solver = 'lbfgs'
        
        model = LogisticRegression(
            C=lr_C,
            penalty=lr_penalty,
            solver=lr_solver,
            random_state=RANDOM_STATE,
            n_jobs=-1
        )

        client.log_param(child_run.info.run_id, "lr_C", lr_C)
        client.log_param(child_run.info.run_id, "lr_penalty", lr_penalty)
        client.log_param(child_run.info.run_id, "lr_solver", lr_solver)


    if model_type == 'decision-tree':
        dt_max_depth = trial.suggest_int('dt_max_depth', 5, x_train.shape[1])
        dt_criterion = trial.suggest_categorical("dt_criterion", ['gini', 'entropy'])
        dt_max_leaf_nodes = trial.suggest_int("dt_max_leaf_nodes", 2, 10)
        
        model = DecisionTreeClassifier(
            max_depth=dt_max_depth,
            criterion=dt_criterion,
            max_leaf_nodes=dt_max_leaf_nodes,
            random_state=RANDOM_STATE
          )
    
        client.log_param(child_run.info.run_id, "dt_max_depth", dt_max_depth)
        client.log_param(child_run.info.run_id, "dt_criterion", dt_criterion)
        client.log_param(child_run.info.run_id, "dt_max_leaf_nodes", dt_max_leaf_nodes)


    if model_type == 'extra_tree':
        etc_n_estimators = trial.suggest_int('etc_n_estimators', 10, 5000)
        etc_max_depth = trial.suggest_int('etc_max_depth', 5, x_train.shape[1])
        etc_min_samples_split = trial.suggest_float('etc_min_samples_split', 0.05, 0.2)
        etc_min_samples_leaf = trial.suggest_float('etc_min_samples_leaf', 0.05, 0.2)
        etc_criterion = trial.suggest_categorical("etc_criterion", ['gini', 'entropy'])
        etc_max_leaf_nodes = trial.suggest_int("etc_max_leaf_nodes", 2, 10)
        
        model = ExtraTreesClassifier(
            n_estimators=etc_n_estimators,
            max_depth=etc_max_depth,
            min_samples_split=etc_min_samples_split,
            min_samples_leaf=etc_min_samples_leaf,
            criterion=etc_criterion,
            max_leaf_nodes=etc_max_leaf_nodes,
            random_state=RANDOM_STATE,
            n_jobs=-1
          )
    
        client.log_param(child_run.info.run_id, "etc_n_estimators", etc_n_estimators)
        client.log_param(child_run.info.run_id, "etc_max_depth", etc_max_depth)
        client.log_param(child_run.info.run_id, "etc_min_samples_split", etc_min_samples_split)
        client.log_param(child_run.info.run_id, "etc_min_samples_leaf", etc_min_samples_leaf)
        client.log_param(child_run.info.run_id, "etc_criterion", etc_criterion)
        client.log_param(child_run.info.run_id, "etc_max_leaf_nodes", etc_max_leaf_nodes)


    if model_type == 'randomforest-classifier':
        rfc_n_estimators = trial.suggest_int('rfc_n_estimators', 10, 5000)
        rfc_max_depth = trial.suggest_int('rfc_max_depth', 5, x_train.shape[1])
        rfc_min_samples_split = trial.suggest_float('rfc_min_samples_split', 0.05, 0.2)
        rfc_min_samples_leaf = trial.suggest_float('rfc_min_samples_leaf', 0.05, 0.2)
        rfc_criterion = trial.suggest_categorical("rfc_criterion", ['gini', 'entropy'])
        rfc_max_leaf_nodes = trial.suggest_int("rfc_max_leaf_nodes", 2, 10)
        
        model = RandomForestClassifier(
            n_estimators=rfc_n_estimators,
            max_depth=rfc_max_depth,
            min_samples_split=rfc_min_samples_split,
            min_samples_leaf=rfc_min_samples_leaf,
            criterion=rfc_criterion,
            max_leaf_nodes=rfc_max_leaf_nodes,
            random_state=RANDOM_STATE,
            n_jobs=-1
          )
    
        client.log_param(child_run.info.run_id, "rfc_n_estimators", rfc_n_estimators)
        client.log_param(child_run.info.run_id, "rfc_max_depth", rfc_max_depth)
        client.log_param(child_run.info.run_id, "rfc_min_samples_split", rfc_min_samples_split)
        client.log_param(child_run.info.run_id, "rfc_min_samples_leaf", rfc_min_samples_leaf)
        client.log_param(child_run.info.run_id, "rfc_criterion", rfc_criterion)
        client.log_param(child_run.info.run_id, "rfc_max_leaf_nodes", rfc_max_leaf_nodes)


    if model_type == 'xgb':
        xgb_n_estimators = trial.suggest_int('xgb_n_estimators', 10, 5000)
        xgb_learning_rate = trial.suggest_float("xgb_learning_rate", 0.01, 1, log=True)
        xgb_reg_lambda = trial.suggest_float("xgb_reg_lambda", 1e-9, 100, log=True)
        xgb_reg_alpha = trial.suggest_float("xgb_reg_alpha", 1e-9, 100, log=True)
        
        model = XGBClassifier(
            n_estimators=xgb_n_estimators,
            learning_rate=xgb_learning_rate,
            reg_lambda=xgb_reg_lambda,
            reg_alpha=xgb_reg_alpha,
            random_state=RANDOM_STATE,
            n_jobs = -1
          )
    
        client.log_param(child_run.info.run_id, "xgb_n_estimators", xgb_n_estimators)
        client.log_param(child_run.info.run_id, "xgb_learning_rate", xgb_learning_rate)
        client.log_param(child_run.info.run_id, "xgb_reg_lambda", xgb_reg_lambda)
        client.log_param(child_run.info.run_id, "xgb_reg_alpha", xgb_reg_alpha)


    if model_type == 'lgb':
        lgb_n_estimators = trial.suggest_int('lgb_n_estimators', 10, 5000)
        lgb_learning_rate = trial.suggest_float("lgb_learning_rate", 0.01, 10, log=True)
        lgb_max_depth = trial.suggest_int('lgb_max_depth', 1, 16)
        lgb_num_leaves = trial.suggest_int('lgb_num_leaves', 2, 2^lgb_max_depth+3)
        lgb_min_data_in_leaf = trial.suggest_int('lgb_min_data_in_leaf', 0, 300)
        lgb_subsample = trial.suggest_float('lgb_subsample', 0.01, 1)
        lgb_feature_fraction = trial.suggest_float('lgb_feature_fraction', 0.1, 1)
        lgb_reg_lambda = trial.suggest_float("lgb_reg_lambda", 0.01, 100, log=True)
        lgb_reg_alpha = trial.suggest_float("lgb_reg_alpha", 0.01, 100, log=True)
        
        model = LGBMClassifier(
            n_estimators=lgb_n_estimators,
            learning_rate=lgb_learning_rate,
            max_depth=lgb_max_depth,
            num_leaves=lgb_num_leaves,
            min_data_in_leaf=lgb_min_data_in_leaf,
            subsample=lgb_subsample,
            feature_fraction=lgb_feature_fraction,
            reg_lambda=lgb_reg_lambda,
            reg_alpha=lgb_reg_alpha,
            # num_threads = 4,
            random_state=RANDOM_STATE,
            n_jobs=-1
            # device_type='cuda_exp'
          )
    
        client.log_param(child_run.info.run_id, "lgb_n_estimators", lgb_n_estimators)
        client.log_param(child_run.info.run_id, "lgb_learning_rate", lgb_learning_rate)
        client.log_param(child_run.info.run_id, "lgb_max_depth", lgb_max_depth)
        client.log_param(child_run.info.run_id, "lgb_num_leaves", lgb_num_leaves)
        client.log_param(child_run.info.run_id, "lgb_min_data_in_leaf", lgb_min_data_in_leaf)
        client.log_param(child_run.info.run_id, "lgb_subsample", lgb_subsample)
        client.log_param(child_run.info.run_id, "lgb_feature_fraction", lgb_feature_fraction)
        client.log_param(child_run.info.run_id, "lgb_reg_lambda", lgb_reg_lambda)
        client.log_param(child_run.info.run_id, "lgb_reg_alpha", lgb_reg_alpha)


    if model_type == 'gradient-boosting':
        gbc_n_estimators = trial.suggest_int('gbc_n_estimators', 10, 5000)
        gbc_learning_rate = trial.suggest_float("gbc_learning_rate", 0.01, 10, log=True)
        gbc_max_depth = trial.suggest_int('gbc_max_depth', 1, 16)
        gbc_min_samples_split = trial.suggest_float('gbc_min_samples_split', 0.05, 0.2)
        gbc_min_samples_leaf = trial.suggest_float('gbc_min_samples_leaf', 0.05, 0.2)
        gbc_max_leaf_nodes = trial.suggest_int("gbc_max_leaf_nodes", 2, 10)
        
        model = GradientBoostingClassifier(
            n_estimators=gbc_n_estimators,
            learning_rate=gbc_learning_rate,
            max_depth=gbc_max_depth,
            min_samples_split=gbc_min_samples_split,
            min_samples_leaf=gbc_min_samples_leaf,
            max_leaf_nodes=gbc_max_leaf_nodes,
            random_state=RANDOM_STATE,
          )
    
        client.log_param(child_run.info.run_id, "gbc_n_estimators", gbc_n_estimators)
        client.log_param(child_run.info.run_id, "gbc_learning_rate", gbc_learning_rate)
        client.log_param(child_run.info.run_id, "gbc_max_depth", gbc_max_depth)
        client.log_param(child_run.info.run_id, "gbc_min_samples_split", gbc_min_samples_split)
        client.log_param(child_run.info.run_id, "gbc_min_samples_leaf", gbc_min_samples_leaf)
        client.log_param(child_run.info.run_id, "gbc_max_leaf_nodes", gbc_max_leaf_nodes)


    ''' create pipeline '''
    client.log_param(child_run.info.run_id, "algo", model.__class__.__name__)

    if feature_selection == "reduce":
        pipeline = Pipeline(steps=[
            ('columnprep', col_transform),
            # ('selector', VarianceThreshold(threshold=(.95 * (1 - .95)))),
            ('reducedim', PCA(n_components=reducedim__n_components)),
            ('algo', model)
        ])

    else:
        pipeline = Pipeline(steps=[
            ('columnprep', col_transform),
            # ('reducedim', PCA(n_components=reducedim__n_components)),
            ('algo', model)
        ])

    # save time with pruning
    # if an experiment seems unpromising based on some intermediate values of loss or validation metric
    # the experiment is discontinued
    # if trial.should_prune():
    #         raise optuna.TrialPruned()
            
    return pipeline, child_run

In [7]:
def evaluate_model(x_train, y_train, y_validate, y_validate_pred, y_validate_scores, pipeline, child_run):
    """
    evaluate the classification model with
    - classification report
    - precision-recall-curve
    - ROC curve
    """

    def plot_learning_curve(pipeline, x_train, y_train):

        train_sizes, train_scores, test_scores = learning_curve(
            pipeline,
            x_train,
            y_train,
            cv=5,
            n_jobs=-1,
            train_sizes=np.linspace(.1, 1.0, 8)
            )


        fig1, ax1 = plt.subplots()
        ax1.set_xlabel("Training examples")
        ax1.set_ylabel("Score")
        train_scores_mean = np.mean(train_scores, axis=1)
        train_scores_std = np.std(train_scores, axis=1)
        test_scores_mean = np.mean(test_scores, axis=1)
        test_scores_std = np.std(test_scores, axis=1)
        ax1.grid()

        ax1.fill_between(train_sizes, train_scores_mean - train_scores_std,
                        train_scores_mean + train_scores_std, alpha=0.1,
                        color="r")
        ax1.fill_between(train_sizes, test_scores_mean - test_scores_std,
                        test_scores_mean + test_scores_std, alpha=0.1, color="g")
        ax1.plot(train_sizes, train_scores_mean, 'o-', color="r",
                label="Training score")
        ax1.plot(train_sizes, test_scores_mean, 'o-', color="g",
                label="Cross-validation score")

        ax1.legend(loc="best")
        ax1.set_title("Difference between training and CV: "\
            + str(round(test_scores_mean[7] / train_scores_mean[7] * 100, 2))\
            + "%")
        client.log_figure(child_run.info.run_id, fig1, 'plot_learning_curve.png')
        plt.close()


    def plot_confusion_matrix(y_validate, y_validate_pred):
        group_names = ["True Neg", "False Pos", "False Neg", "True Pos"]
        group_counts = ["{0:0.0f}".format(value) for value in
                        confusion_matrix(y_validate, y_validate_pred).flatten()]
        group_percentages = ["{0:.2%}".format(value) for value in
                             confusion_matrix(y_validate, y_validate_pred).flatten()/np.sum(confusion_matrix(y_validate, y_validate_pred))]
        labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in
                  zip(group_names,group_counts,group_percentages)]
        labels = np.asarray(labels).reshape(2,2)

        fig2, ax2 = plt.subplots()
        sns.heatmap(confusion_matrix(y_validate, y_validate_pred), annot=labels, fmt="", cmap='Blues')
        plt.xlabel('Predicted Label')
        plt.ylabel('True Label')
        client.log_figure(child_run.info.run_id, fig2, 'plot_confusion_matrix.png')
        plt.close()


    def plot_precision_recall_vs_threshold(y_validate, y_scores, child_run):
        precisions, recalls, thresholds = precision_recall_curve(y_validate, y_scores)

        # convert to f score
        fscore = (2 * precisions * recalls) / (precisions + recalls)
        # locate the index of the largest f score
        ix = np.argmax(fscore)
        
        client.log_metric(child_run.info.run_id, "f1_score", round(fscore[ix], 5))

        fig3, ax3 = plt.subplots()
        ax3.plot(thresholds, precisions[:-1], "b", label="Precision")
        ax3.plot(thresholds, recalls[:-1], "g", label="Recall")
        ax3.plot(thresholds, fscore[:-1], "r", label="F1 Score")
        ax3.axvline(x=thresholds[ix], color='red', linestyle='--')
        plt.axhline(y=precisions[ix], color='b', linestyle='--')
        plt.axhline(y=recalls[ix], color='g', linestyle='--')
        ax3.set_xlabel("Threshold")
        ax3.legend(loc="upper left")
        ax3.set_ylim([0,1])
        client.log_figure(child_run.info.run_id, fig3, 'plot_f1.png')
        plt.close()

        fig4, ax4 = plt.subplots()
        ax4.plot(recalls, precisions, marker='.', label='Logistic')
        ax4.scatter(recalls[ix], precisions[ix], 200, marker='o', color='red', label='Best')
        ax4.set_xlabel('Recall')
        ax4.set_ylabel('Precision')
        client.log_figure(child_run.info.run_id, fig4, 'plot_precision_recall.png')
        plt.close()
        

    def plot_roc_curve(y_validate, y_scores, child_run):
        fpr, tpr, thresholds = roc_curve(y_validate, y_scores)

        roc_auc = round(roc_auc_score(y_validate, y_scores), 3)
        
        optimal_idx = np.argmax(tpr - fpr)

        fig5, ax5 = plt.subplots()
        ax5.plot(fpr, tpr, linewidth=2)
        ax5.plot([0,1], [0,1], 'k--')
        ax5.axis([0,1,0,1])
        ax5.scatter(fpr[optimal_idx], tpr[optimal_idx], 200, marker='o', color='red', label='Best')
        ax5.set_xlabel('False Positive Rate')
        ax5.set_ylabel('True Positive Rate')
        client.log_figure(child_run.info.run_id, fig5, 'plot_roc_curve.png')
        plt.close()

        client.log_metric(child_run.info.run_id, "roc_auc", roc_auc)

        

    plot_confusion_matrix(y_validate, y_validate_pred)
    plot_precision_recall_vs_threshold(y_validate, y_validate_scores, child_run)
    plot_roc_curve(y_validate, y_validate_scores, child_run)
    plot_learning_curve(pipeline, x_train, y_train)

In [8]:
class Objective:
    
    def __init__(self):
        self.best_model = None
        self._model = None

    
    def __call__(self, trial):
    
        pipeline, child_run = create_model(trial)

        score = cross_val_score(
            pipeline,
            x_train,
            y_train,
            cv=cv,
            scoring="accuracy",
            n_jobs=-1
        ).mean()

        client.log_metric(child_run.info.run_id, "cv_score", score)


        # fit the pipeline to compute the validation results
        pipeline.fit(x_train, y_train)
        self._model = pipeline

        # predict the training outcome
        y_validate_pred = pipeline.predict(x_validate)

        # predict probabilities
        y_validate_proba = pipeline.predict_proba(x_validate)
        # keep probabilities for the positive outcome only
        y_validate_scores = y_validate_proba[:, 1]

        evaluate_model(x_train, y_train, y_validate, y_validate_pred, y_validate_scores, pipeline, child_run)


        return score

    def callback(self, study, trial):
        if study.best_trial == trial:
            self.best_model = self._model


In [9]:
def create_submission(best_model, x_test, name=None):
    # predict the test values with the training classification model
    y_pred = best_model.predict(x_test).astype(int)
    
    df_submission = pd.read_csv("../01_rawdata/gender_submission.csv")
    df_submission['Survived'] = y_pred
    
    if name == None:
        df_submission.to_csv('submissions/%s.csv'%parent_run.info.run_id, index=False)
    else:
        df_submission.to_csv('submissions/%s.csv'%name, index=False)


In [10]:
objective = Objective()

study = optuna.create_study(
  pruner=optuna.pruners.MedianPruner(n_startup_trials=1, n_warmup_steps=2, interval_steps=3),
  sampler = optuna.samplers.TPESampler(),
  direction="maximize"
  )

study.optimize(
  objective,
  n_trials=N_TRAILS,
  timeout=TIMEOUT,
  n_jobs=-1,
  callbacks=[objective.callback]
  )

pruned_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.PRUNED]
complete_trials = [t for t in study.trials if t.state == optuna.trial.TrialState.COMPLETE]

print("Study statistics: ")
print("Number of finished trials: ", len(study.trials))
print("Number of pruned trials: ", len(pruned_trials))
print("Number of complete trials: ", len(complete_trials))

print("Best trial:")
print(study.best_value)
print(study.best_params)

client.log_metric(parent_run.info.run_id, "best_cv_score", round(study.best_value, 3))
client.log_param(parent_run.info.run_id, "transformer_num", str(transformer_num))

client.log_param(parent_run.info.run_id, "cv_n_splits", cv.n_splits)
client.log_param(parent_run.info.run_id, "cv_train_size", cv.train_size)
client.log_param(parent_run.info.run_id, "cv_test_size", cv.test_size)
client.log_param(parent_run.info.run_id, "cv_random_state", cv.random_state)

for param in study.best_params:
  client.log_param(parent_run.info.run_id, param, study.best_params[param])


# save the best model as file
best_model = objective.best_model
mlflow.sklearn.save_model(best_model, "models/%s/"%parent_run.info.run_id)

# create submission of best model
create_submission(best_model, x_test)


mlflow.end_run()

[I 2022-09-19 20:53:40,622] A new study created in memory with name: no-name-b6b61271-1b37-4ff5-b04b-4360e2e71fbb
[I 2022-09-19 20:53:48,326] Trial 0 finished with value: 0.7989304812834225 and parameters: {'feature_selection': 'reduce', 'columnprep__transformers_num': 'StandardScaler', 'reducedim__n_components': 0.8522825529733893, 'model_type': 'decision-tree', 'dt_max_depth': 21, 'dt_criterion': 'gini', 'dt_max_leaf_nodes': 10}. Best is trial 0 with value: 0.7989304812834225.
[I 2022-09-19 20:54:04,353] Trial 3 finished with value: 0.827807486631016 and parameters: {'feature_selection': 'none', 'columnprep__transformers_num': 'MinMaxScaler', 'reducedim__n_components': 0.8491907372057212, 'model_type': 'xgb', 'xgb_n_estimators': 2526, 'xgb_learning_rate': 0.3100892201891197, 'xgb_reg_lambda': 30.52564401458145, 'xgb_reg_alpha': 10.418247753290977}. Best is trial 3 with value: 0.827807486631016.
[I 2022-09-19 20:54:58,737] Trial 4 finished with value: 0.7518716577540107 and parameters

Study statistics: 
Number of finished trials:  50
Number of pruned trials:  0
Number of complete trials:  50
Best trial:
0.8374331550802138
{'feature_selection': 'none', 'columnprep__transformers_num': 'StandardScaler', 'reducedim__n_components': 0.8086671801121819, 'model_type': 'logistic-regression', 'lr_C': 1.122451275410539, 'lr_penalty': 'l1'}


In [11]:
mlflow.end_run()

In [12]:
optuna.visualization.plot_optimization_history(study)